In [2]:

import keras_tuner as kt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from preprocessing import get_preprocessor
import matplotlib.pyplot as plt

# Cargar datos
housing = pd.read_csv("housing.csv")
X = housing.drop("median_house_value", axis=1)
y = housing["median_house_value"].copy()

# División
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Escalar y
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_valid_scaled = y_scaler.transform(y_valid.values.reshape(-1, 1))

# Preprocesamiento
preprocessor = get_preprocessor(housing)
X_train_prep = preprocessor.fit_transform(X_train)
X_valid_prep = preprocessor.transform(X_valid)

# Definir modelo con hiperparámetros
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=X_train_prep.shape[1]))

    for i in range(hp.Int("num_layers", 2, 5)):
        model.add(keras.layers.Dense(units=hp.Int(f"units_{i}", min_value=64, max_value=512, step=64),
                                     activation="relu"))
        if hp.Boolean(f"batch_norm_{i}"):
            model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dropout(rate=hp.Float(f"dropout_{i}", 0.1, 0.5, step=0.1)))

    model.add(keras.layers.Dense(1))
    model.compile(optimizer=keras.optimizers.Adam(
                    hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")),
                  loss="mse")
    return model

# Instancia de tuner
tuner = kt.RandomSearch(
    build_model,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="kt_dir",
    project_name="california_housing_nn"
)

# Búsqueda
tuner.search(X_train_prep, y_train_scaled, epochs=100,
             validation_data=(X_valid_prep, y_valid_scaled),
             callbacks=[keras.callbacks.EarlyStopping(patience=10)])

# Mejor modelo
best_model = tuner.get_best_models(1)[0]

# Evaluación
y_valid_pred_scaled = best_model.predict(X_valid_prep)
y_valid_pred = y_scaler.inverse_transform(y_valid_pred_scaled)
rmse_valid = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
print("✅ RMSE validación (mejor modelo KerasTuner):", rmse_valid)


Trial 8 Complete [00h 00m 17s]
val_loss: 0.27373987436294556

Best val_loss So Far: 0.23580367863178253
Total elapsed time: 00h 05m 17s

Search: Running Trial #9

Value             |Best Value So Far |Hyperparameter
4                 |4                 |num_layers
320               |512               |units_0
True              |True              |batch_norm_0
0.1               |0.2               |dropout_0
128               |64                |units_1
True              |True              |batch_norm_1
0.5               |0.3               |dropout_1
0.00032253        |0.0062843         |learning_rate
64                |64                |units_2
False             |False             |batch_norm_2
0.1               |0.1               |dropout_2
320               |64                |units_3
False             |False             |batch_norm_3
0.3               |0.1               |dropout_3
128               |None              |units_4
False             |None              |batch_norm_4
0.5   

KeyboardInterrupt: 

In [ ]:

# Preprocesamiento del conjunto de test
X_test_prep = preprocessor.transform(X_test)
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

# Predicción con el mejor modelo
y_test_pred_scaled = best_model.predict(X_test_prep)
y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled)

# Evaluación
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print("✅ RMSE test (mejor modelo KerasTuner):", rmse_test)

# Guardar predicciones
pd.DataFrame({
    "y_real": y_test,
    "y_pred": y_test_pred.flatten()
}).to_csv("predicciones_test_kerastuner.csv", index=False)
